In [2]:
!pip install xarray numpy pandas tqdm
import xarray as xr, numpy as np, pandas as pd, numpy.linalg as la
from tqdm.notebook import tqdm
from datetime import timedelta

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------------- -------------- 0.8/1.2 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/12.6 MB 4.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/12.6 MB 4.0 MB/s eta 0:00:03
   ----------- ---------------------------- 3.7/12.6 MB 4.3 MB/s eta 0:00:03
   -------------- ------------------------- 4.5/12.6 MB 4.3 MB/s eta 0:00:02
   ----------------- ---------------------- 5.5/12.6 MB 4.3 MB/s eta 0:00:02
   ------------------- -------------------- 6.0/12.6 MB 4.1 MB/s eta 0:00:02
   -------------------- ------------------- 6.6/12.6 MB 4.0 MB/s eta 0:00:02
   ----------------------- ---------------- 7.3/12.6 MB 3.9 MB/s eta 0:00:02
   -------------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install pyarrow

   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.2 MB 1.6 MB/s eta 0:00:15
   -- ------------------------------------- 1.3/25.2 MB 1.8 MB/s eta 0:00:14
   -- ------------------------------------- 1.6/25.2 MB 1.8 MB/s eta 0:00:14
   -- ------------------------------------- 1.8/25.2 MB 1.8 MB/s eta 0:00:14
   --- ------------------------------------ 2.4/25.2 MB 1.8 MB/s eta 0:00:13
   ---- ----------------------------------- 2.6/25.2 MB 1.8 MB/s eta 0:00:13
   ---- ----------------------------------- 3.1/25.2 MB 1.8 MB/s eta 0:00:13
   ----- ---------------------------------- 3.4/25.2 MB 1.8 MB/s eta 0:00:12
   ------ --------------------------------- 3.9/25.2 MB 1.9 MB/s eta 0:00:12
   ------ --------------------------------- 4.2/25.2 MB 1.9 MB/s eta 0:00:12
   ------- ---------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
def interpolate_station(lat, lon, lats, lons, values):
    lon = -lon
    N = np.flatnonzero(lat >= lats)[0]
    n0, n1 = N-1, N
    E = np.flatnonzero(lon <= lons)[0]
    e0, e1 = E-1, E
    assert(0 < N < len(lats))
    assert(0 < E < len(lons))
    neargrid = np.array([
        [lats[n0], lats[n1], lats[n0], lats[n1]],
        [lons[e0], lons[e0], lons[e1], lons[e1]]])
    nearvec = neargrid - np.array([[lat],[lon]])
    nearvec[1] = nearvec[1]*np.cos(np.radians(65)) # correct for unequal lat and lon
    dist = la.norm(nearvec, axis=0)
    weights = 1/np.maximum(dist,0.000001)
    weights /= sum(weights)
    values_near = values[:, N-1:N+1, E-1:E+1].reshape(len(values), 4)
    result = values_near @ weights
    return result

In [ ]:
# Read stations and data-file
station_data = pd.read_csv('../../../data/Vedur/stod.txt')
station_data = station_data[['stod', 'breidd', 'lengd']]
data = pd.read_feather("../../../data/combined/data.feather")
data["lat"]
data = data[["stod", "time"]]
data = pd.merge(data, station_data, on = "stod")

KeyError: "None of [Index(['stod', 'time'], dtype='object')] are in the [columns]"

In [19]:
data.columns[:15]

Index(['gust_factor', 'ws_15', 'wd_15', 't_15', 'p_15', 'Ri', 'N_squared',
       'station_elevation', 'X', 'Y', 'twd', 'elevation_0', 'elevation_1',
       'elevation_2', 'elevation_3'],
      dtype='object')